<a href="https://colab.research.google.com/github/Nim-ah/MIE1614_Naive_Bias/blob/main/Submissions/Task3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3

- Intro to task 3
- Used connection_df (Uoft_edges.csv)
- features Task 3c visualization

In [5]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install 'scipy>=1.8'
!pip install pyvis
!pip install --upgrade scipy networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import networkx as nx
from torch_geometric.loader import DataLoader
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Loading

In [7]:
task_2_df = pd.read_csv('NEW_features_w_customer_id.csv').rename(columns={"CUSTOMER_ID":"customer_id"}) # extracted from katia_xgboost.ipynb
task_2_df["customer_id"] = task_2_df["customer_id"].astype(int)

In [8]:
task_2_df

,Unnamed: 0,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,...,WIRES_CNT_OUT,customer_id,code,CUST_AGE,ACCNT_AGE,ACCNT_OPEN_LST_YEAR,COUNTRY_RISK_INCOME_CODE,COUNTRY_RISK_RESIDENCY_CODE,RISK_CODE,OCPTN_RISK_CODE
0,0,0.120507,0.107623,-0.328663,-0.614327,-0.782851,-0.614963,-0.773760,-0.580594,-0.350359,...,-0.977304,935382,89,-0.520973,-0.291996,1.0,-0.057616,-0.099080,0,1.350398
1,1,0.120507,0.107623,-0.328663,-0.591805,-0.782851,-0.556341,-0.390381,-0.543728,-0.140694,...,-0.003588,2305,89,-1.205196,-1.489772,1.0,-0.057616,-0.099080,0,1.350398
2,2,0.120507,0.107623,-0.328663,-0.661574,-0.901526,-0.596031,-0.582071,-0.591566,-1.189021,...,-1.102139,472403,89,0.509868,-0.703317,1.0,-0.057616,-0.099080,0,1.350398
3,3,0.120507,0.107623,-0.328663,-0.564667,-0.426824,-0.642281,-0.725838,-0.566957,-0.215574,...,0.712135,227994,89,-1.442482,0.301919,1.0,-0.057616,-0.099080,0,1.350398
4,4,0.120507,0.107623,-0.328663,-0.656262,-0.782851,-0.304639,0.951445,-0.576850,0.158828,...,0.229438,418265,89,1.651392,-0.828454,1.0,-0.057616,-0.099080,0,1.350398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999470,-8.298289,0.107623,-0.328663,-0.257978,-0.189473,0.901319,0.615988,1.042289,-0.185622,...,-0.419706,701136,841,-0.172403,0.594181,1.0,-0.057616,5.090803,1,2.395387
999996,999676,-8.298289,-9.291725,-0.328663,0.609053,0.285229,-0.186069,-0.534149,-0.061281,-1.054236,...,-0.902402,220596,841,-0.377250,-1.797503,0.0,-0.057616,5.090803,1,2.395387
999997,999748,-8.298289,0.107623,-0.328663,-0.371050,-0.664175,1.377776,1.095212,2.647045,-0.005909,...,0.928516,89038,841,0.679272,0.138938,1.0,-0.057616,10.280686,1,2.395387
999998,999754,-8.298289,0.107623,-0.328663,1.123092,0.878606,-0.617623,-1.013372,0.125550,-0.440216,...,-0.652732,805515,841,-0.614986,-0.941436,1.0,-0.057616,10.280686,1,2.395387


In [9]:
connections_data = pd.read_csv("UofT_edges.csv")
connections_df = pd.DataFrame(connections_data).sort_values(by="source").rename(columns={"source":"customer_id"})
connections_df['customer_id'] = connections_df['customer_id'].astype(int)
connections_df['target'] = connections_df['target'].astype(int)
connections_df

,customer_id,target,emt
32127,7,69883,509.911692
32147,7,980973,4984.327973
32144,7,314999,2984.836484
217572,11,876639,2910.499409
450974,25,579365,311.526741
...,...,...,...
429432,1000672,14024,114.625882
463894,1000673,295624,161.419638
146306,1000674,721075,2952.622741
146297,1000674,595097,1479.290194


In [10]:
uniq_source = np.unique(connections_df["customer_id"].values)
uniq_target = np.unique(connections_df["target"].values)

total_uniq = np.concatenate((uniq_source, uniq_target))

In [11]:
print("Total Amount of unique nodes in connection_df (Uoft_edges.csv):", len(np.unique(total_uniq)))

Total Amount of unique nodes in connection_df (Uoft_edges.csv): 361178


In [12]:
mapping = {}
for i, cust_id in enumerate(task_2_df["customer_id"].values):
    mapping[cust_id] = i

In [13]:
task_2_df['customer_id']=task_2_df.customer_id.map(mapping)
connections_df['customer_id']=connections_df.customer_id.map(mapping)
connections_df['target']=connections_df.target.map(mapping)

### Verify graph data nodes and edges in pytorch geometric form:

x – Node feature matrix [customer_id, age, risk, etc..]

edge_index – edge connections (source to target) [2, num_edges] 

edge_attr – money transfers (dollars) [num_edges, num_edge_features]

In [14]:
x = torch.tensor(task_2_df[["customer_id", "COUNTRY_RISK_RESIDENCY_CODE"]].values)

In [15]:
edge_index = torch.tensor(connections_df[["customer_id", "target"]].values.T)

In [16]:
edge_attr = torch.tensor(connections_df["emt"].values)

In [17]:
from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

### Data Verification

In [18]:
data.num_nodes

1000000

In [19]:
data.validate()

True

In [20]:
data.is_directed()

True

In [21]:
loader = DataLoader(data, batch_size=32, shuffle=True)

In [22]:
data.edge_attr

tensor([ 509.9117, 4984.3280, 2984.8365,  ..., 2952.6227, 1479.2902,
        2997.5295], dtype=torch.float64)

In [23]:
max_edge = data.edge_attr.max()
min_edge = data.edge_attr.min()
mean_edge = data.edge_attr.mean()

In [24]:
print("EMT: Max = {}, Min = {}, Mean = {}".format(max_edge, min_edge, mean_edge))

EMT: Max = 5394.814418136593, Min = 56.91276323887504, Mean = 1080.3400282148295


In [25]:
risk_id = task_2_df[["customer_id", "COUNTRY_RISK_RESIDENCY_CODE"]]
risk_id["source"] = task_2_df["customer_id"]

<ipython-input-25-c29f18407e08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  risk_id["source"] = task_2_df["customer_id"]


# Created function for tree depth in Connection Dataframe

In [26]:
def count_number_of_people_in_tree(origin_id, level_depth):
    """
    Count number of people in a tree (searches down the tree) for a given customer id and max level depth
    """
    list_id = [origin_id]
    level = 1
    df_sub_graphs = []

    while level <= level_depth:
        all_sub_level_targets = []
        for cust_id in list_id:
            sub_level_targets = connections_data[connections_data["source"]==cust_id]["target"].tolist()
            emts = connections_data[connections_data["source"]==cust_id]["emt"].tolist()
            data = np.array([[cust_id] * len(sub_level_targets), sub_level_targets, [level]*len(sub_level_targets), emts]).T
            df_sub_graphs.append(pd.DataFrame(columns=["source", "target", "level", "emt"], data=data))
            all_sub_level_targets += sub_level_targets
        level += 1
        list_id = all_sub_level_targets

    return pd.concat(df_sub_graphs)#["target"].unique().shape[0]

In [27]:
source_id = connections_data.source

In [28]:
risk_id["COUNTRY_RISK_RESIDENCY_CODE"].unique()

array([-0.09908006,  5.0908031 , 10.28068627])

In [29]:
temp = count_number_of_people_in_tree(origin_id=983859.0, level_depth=10)
temp = temp.reset_index()

In [30]:
merged_df = pd.merge(temp, risk_id, on='source', how='inner')
merged_df["COUNTRY_RISK_RESIDENCY_CODE"].unique()


array([-0.09908006])

In [34]:
# for source in source_id:
#   temp = count_number_of_people_in_tree(origin_id=source, level_depth=10)
#   temp = temp.reset_index()
#   merged_df = pd.merge(temp, risk_id, on='source', how='inner')
#   num = merged_df["COUNTRY_RISK_RESIDENCY_CODE"].unique()
  
#   if len(num) >= 2:
#     print("", source)

# Graph Visualization (NetworkX/pyvis)
- Visualized EMT branching from desired Source to Target

In [ ]:
new_source_id = [7.0, 999042, 998874.0]

In [ ]:
from pyvis.physics import Physics
from pyvis.network import Network

for i, source in enumerate(new_source_id):
  if i > 2:
      break
  temp = count_number_of_people_in_tree(origin_id=source, level_depth=10)
  temp = temp.reset_index()
  merged_df = pd.merge(temp, risk_id, on='source', how='inner')
  source_data = merged_df["source"]
  target_data = merged_df["target"]
  emt_data = merged_df["emt"]
  risk_data = merged_df["COUNTRY_RISK_RESIDENCY_CODE"]
  level_data = merged_df["level"]
  names = []
  num_list = [len(merged_df)]

  for num in num_list:
    G = nx.Graph()
    for row in range(num):
      node1 = int(source_data[row])
      node2 = int(target_data[row])
      emt = int(emt_data[row])
      color = int(risk_data[row])
      G.add_node(node1, risk=color, label=node1)
      G.add_node(node2, risk=color, label=node2)
      G.add_edge(node1, node2)
      names.append((node1,node2))

    # Load the graph object into the Network object
    G.add_edges_from(names)
    G2 = Network(directed=True)
    G2.from_nx(G)
    
    color_map = nx.get_node_attributes(G, 'risk')
    neighbor_map = G2.get_adj_list()
    
    for key in color_map:
      if key == source:
        color_map[key] = 'purple'
      elif color_map[key] == int(np.unique(risk_data)[0]):
        color_map[key] = 'green'
      elif color_map[key] == int(np.unique(risk_data)[1]):  
        color_map[key] = 'orange'
      elif color_map[key] == int(np.unique(risk_data)[2]):  
        color_map[key] = 'red'  

    for node in G2.nodes:
        node["value"] = len(neighbor_map[node["id"]])  #larger node: more neighbours
        node["color"] = color_map[node['id']]
        node["label"] = str(node['id'])

    G2.inherit_edge_colors(True)
    # Show the visualization
    name = "/content/nx{}.html".format(i)
    #G2.show_buttons(filter_=['physics'])
    G2.force_atlas_2based()
    G2.show(name, local=True, notebook=False)

### Will save graph data locally 